In [1]:
#@title Configuration { form-width: "40%", display-mode: "form" }
import os
import numpy as np
import random
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras import Sequential, optimizers
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, Dropout, Flatten, BatchNormalization, Conv2D, Input, MaxPool2D


#@markdown ##Determinism
#@markdown ---
ENABLE_DETERMINISM = False #@param {type: "boolean"}
SEED = 17 #@param
AUTO = tf.data.AUTOTUNE
#@markdown ##Batches
#@markdown ---
TRAIN_BATCH_SIZE = 1024 #@param
TEST_BATCH_SIZE = 16 #@param
#@markdown ##Training
#@markdown ---
NUMBER_OF_EPOCHS = 500 #@param
EARLY_STOPPING_TOLERANCE = 10 #@param
LEARNING_RATE = 1e-4 #@param {type:"number"}
IMAGE_SIZE = 128 #@param
SHUFFLE_BUFFER = 300000 #@param {type: "number"}
TPU = True #@param {type: "boolean"}
#@markdown ##Augmentations
#@markdown ---
AUGMENTATIONS_ROTATE = False #@param {type:"boolean"}
AUGMENTATIONS_FLIP_HORIZONTALLY = False #@param {type:"boolean"}
AUGMENTATIONS_FLIP_VERTICALLY = False #@param {type:"boolean"}
AUGMENTATIONS_ZOOM = False #@param {type:"boolean"}
AUGMENTATIONS_RANDOM_NOISE = True #@param {type:"boolean"}
#@markdown ##Paths
#@markdown ---
GALAXY_DATASET = "SDSS" #@param {type:"string"}
EXPERIMENT_DESCRIPTION = "conference/random_zoom_only" #@param {type:"string"}
LOCAL_GCP_PATH_BASE = f"gcs/{GALAXY_DATASET}"
REMOTE_GCP_PATH_BASE = f"gs://tomasmuzas-phd-datasets/{GALAXY_DATASET}"



if ENABLE_DETERMINISM:
  print(f"Enabling determinism. Seed: {SEED}.")
  tf.keras.utils.set_random_seed(SEED)
  tf.config.experimental.enable_op_determinism()
else:
  print("No determinism enabled.")

No determinism enabled.


In [2]:
from google.colab import auth
auth.authenticate_user()

# Dataset

In [3]:
@tf.function
def random_invert_horizontally(x, y, seed):
  return tf.cond(
    tf.random.uniform([], seed = seed) < 0.5,
    lambda: tf.image.flip_left_right(x),
    lambda: x)

@tf.function
def random_invert_vertically(x, y, seed):
  return tf.cond(
    tf.random.uniform([], seed = seed) < 0.5,
    lambda: tf.image.flip_up_down(x),
    lambda: x)

#and tf.math.equal(y, 1) == True:
@tf.function
def random_rotate(x, y, seed):
  return tf.cond(
      tf.random.uniform([], seed = seed) < 0.5,
      lambda: tf.image.rot90(x, k = tf.random.uniform([], minval=1, maxval=4, dtype=tf.int32, seed = seed)),
      lambda: x)

@tf.function
def random_zoom(x, y, seed):
  if  tf.random.uniform([]) < 0.5: #and tf.math.equal(y, 1) == True:
    x = tf.image.crop_to_bounding_box(x, 10, 10, IMAGE_SIZE - 20, IMAGE_SIZE - 20)
    x = tf.image.resize(x, (IMAGE_SIZE, IMAGE_SIZE))
  else:
    x
  return x

@tf.function
def random_noise(x, y, seed):
  if  tf.random.uniform([], seed = seed) < 0.5: #and tf.math.equal(y, 1) == True:
    noise = tf.random.uniform(shape=tf.shape(x), minval= -0.2, maxval= 0.2, dtype=tf.float32)
    x = tf.add(x, noise)
    x = tf.clip_by_value(x, clip_value_min=0, clip_value_max=1)
  else:
    x
  return x

def get_dataset(path, batch_size, seed, augment, shuffle, drop_remainder):
  if not ENABLE_DETERMINISM:
    seed = None

  dataset = tf.data.TFRecordDataset(tf.io.gfile.glob(path + "/*.tfrec"), num_parallel_reads=AUTO) # if TPU else 20)

  if(shuffle):
    dataset = dataset.shuffle(SHUFFLE_BUFFER, seed = seed)

  dataset = dataset.map(lambda records: tf.io.parse_single_example(
      records,
      {
          "image": tf.io.FixedLenFeature([], dtype=tf.string),
          "class": tf.io.FixedLenFeature([], dtype=tf.int64),

          "label": tf.io.FixedLenFeature([], dtype=tf.string),
          "objid": tf.io.FixedLenFeature([], dtype=tf.string),
          "one_hot_class": tf.io.VarLenFeature(tf.float32)
      }),
      num_parallel_calls=AUTO)
  dataset = dataset.map(lambda item: (tf.reshape(tf.image.decode_jpeg(item['image'], channels=3), [IMAGE_SIZE, IMAGE_SIZE, 3]), item['class']), num_parallel_calls=AUTO)
  dataset = dataset.map(lambda x,y: (tf.cast(x, tf.float32), y), num_parallel_calls=AUTO)
  dataset = dataset.map(lambda x,y: (tf.keras.layers.Rescaling(scale=1./255)(x), y), num_parallel_calls=AUTO)

  if(augment):
      if AUGMENTATIONS_ZOOM:
        dataset = dataset.map(lambda x,y : (random_zoom(x, y, seed), y), num_parallel_calls=AUTO)
      if AUGMENTATIONS_FLIP_HORIZONTALLY:
        dataset = dataset.map(lambda x,y : (random_invert_horizontally(x, y, seed), y), num_parallel_calls=AUTO)
      if AUGMENTATIONS_FLIP_VERTICALLY:
        dataset = dataset.map(lambda x,y : (random_invert_vertically(x, y, seed), y), num_parallel_calls=AUTO)
      if AUGMENTATIONS_ROTATE:
        dataset = dataset.map(lambda x,y : (random_rotate(x, y, seed), y), num_parallel_calls=AUTO)
      if AUGMENTATIONS_RANDOM_NOISE:
        dataset = dataset.map(lambda x,y : (random_noise(x, y, seed), y), num_parallel_calls=AUTO)

  dataset = dataset.batch(batch_size, drop_remainder=drop_remainder).prefetch(200 if TPU else 3)

  return dataset

# Dataset visualisation

In [ ]:
# import matplotlib.pyplot as plt



# # with strategy.scope():
# dsss = prepare_dataset_for_epochs(5, f"gs://tomasmuzas-phd-datasets/SDSS/sdss_binary_fold_1/train", BATCH_SIZE, seed = SEED, augment = True, drop_remainder = False)

# f, axarr = plt.subplots(5, 5, sharey=True)
# f.set_figheight(20)
# f.set_figwidth(20)

# for it, batch in enumerate(dsss):
#   for j, image in enumerate(batch[0]):
#     axarr[it][j].imshow(image)

# # for i in range(5):
# #   for j, images in enumerate(dsss):
# #     for k, image in enumerate(images[0]):
# #       axarr[i][k].imshow(image)
# plt.show()

# GPU/TPU setup

In [4]:
def reset_tpu():
  if TPU:
    cluster_resolver = tf.distribute.cluster_resolver.TPUClusterResolver.connect()
    strategy = tf.distribute.TPUStrategy(cluster_resolver)
    print("Running on TPU ", cluster_resolver.master())
    print("REPLICAS: ", strategy.num_replicas_in_sync)
    return strategy
  else:
    gpus = tf.config.list_physical_devices('GPU')
    print(gpus)
    print("Running on GPU.")
    return tf.distribute.MirroredStrategy()

reset_tpu()

Running on TPU  grpc://10.31.17.98:8470
REPLICAS:  8


In [5]:
!echo "deb http://packages.cloud.google.com/apt gcsfuse-bionic main" > /etc/apt/sources.list.d/gcsfuse.list
!curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | apt-key add -
!apt -qq update
!apt -qq install gcsfuse
!mkdir gcs
!gcsfuse --implicit-dirs tomasmuzas-phd-datasets gcs

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  2426  100  2426    0     0  31506      0 --:--:-- --:--:-- --:--:-- 31506
OK
26 packages can be upgraded. Run 'apt list --upgradable' to see them.
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following NEW packages will be installed:
  gcsfuse
0 upgraded, 1 newly installed, 0 to remove and 26 not upgraded.
Need to get 13.3 MB of archives.
After this operation, 30.7 MB of additional disk space will be used.
Selecting previously unselected package gcsfuse.
(Reading database ... 124016 files and directories currently installed.)
Preparing to unpack .../gcsfuse_0.41.10_amd64.deb ...
Unpacking gcsfuse (0.41.10) ...
Setting up gcsfuse (0.41.10) ...
2023/01/10 17:38:53.908559 Start gcsfuse/0.41.10 (Go version go1.18.4) for app "" using moun

# Model definitions

In [6]:
def create_ResNet50():
  model = Sequential()
  model.add(ResNet50(
    input_shape = (IMAGE_SIZE, IMAGE_SIZE, 3),
    include_top = False,
    weights = None,
    pooling = 'avg'))
  model.add(Dense(1, activation='sigmoid'))
  return model

def create_EffNetB7():
  model = Sequential()
  model.add(tf.keras.applications.efficientnet.EfficientNetB7(
    input_shape = (IMAGE_SIZE, IMAGE_SIZE, 3),
    include_top = False,
    weights = None,
    pooling = 'avg'))
  model.add(Dense(1, activation='sigmoid'))
  return model

def Dieleman():
  model = Sequential(name="Dieleman")
  model.add(Input(shape=(IMAGE_SIZE, IMAGE_SIZE, 3)))
  model.add(Conv2D(filters=32, kernel_size=6, activation='relu'))
  model.add(BatchNormalization())
  model.add(MaxPool2D(2))
  model.add(Conv2D(filters=64, kernel_size=5, activation='relu'))
  model.add(BatchNormalization())
  model.add(MaxPool2D(2))
  model.add(Conv2D(filters=128, kernel_size=3, activation='relu'))
  model.add(Conv2D(filters=128, kernel_size=3, activation='relu'))
  model.add(BatchNormalization())
  model.add(MaxPool2D(2))
  model.add(Flatten())
  if ENABLE_DETERMINISM:
    model.add(Dropout(0.5, seed = SEED))
  else:
    model.add(Dropout(0.5))
  model.add(Dense(256,activation='relu'))
  model.add(Dense(256,activation='relu'))
  model.add(Dense(1, activation='sigmoid'))
  return model

def SimpleModel():
  model = Sequential(name="SimpleModel")
  model.add(Input(shape=(IMAGE_SIZE, IMAGE_SIZE, 3)))
  model.add(Conv2D(filters=256, kernel_size=6, activation='relu'))
  model.add(Conv2D(filters=128, kernel_size=5, activation='relu'))
  model.add(Conv2D(filters=16, kernel_size=3, activation='relu'))
  model.add(Flatten())
  model.add(Dense(1, activation='sigmoid'))
  return model

def SimplerModel():
  model = Sequential(name="SimplerModel")
  model.add(Input(shape=(IMAGE_SIZE, IMAGE_SIZE, 3)))
  model.add(Conv2D(filters=128, kernel_size=6, activation='relu'))
  model.add(MaxPool2D(2))
  model.add(Conv2D(filters=32, kernel_size=5, activation='relu'))
  model.add(MaxPool2D(2))
  model.add(Conv2D(filters=16, kernel_size=3, activation='relu'))
  model.add(Flatten())
  model.add(Dense(1, activation='sigmoid'))
  return model

def Cavanagh():
  model = Sequential(name="Cavanagh")
  model.add(Input(shape=(IMAGE_SIZE, IMAGE_SIZE, 3)))
  model.add(Conv2D(filters=32, kernel_size=7, activation='relu'))
  model.add(BatchNormalization())
  model.add(MaxPool2D(2))
  model.add(Conv2D(filters=64, kernel_size=5, activation='relu'))
  model.add(Conv2D(filters=64, kernel_size=5, activation='relu'))
  model.add(BatchNormalization())
  model.add(MaxPool2D(2))
  model.add(Conv2D(filters=128, kernel_size=3, activation='relu'))
  model.add(BatchNormalization())
  model.add(MaxPool2D(2))
  model.add(Flatten())
  if ENABLE_DETERMINISM:
    model.add(Dropout(0.5, seed = SEED))
  else:
    model.add(Dropout(0.5))
  model.add(Dense(256,activation='relu'))
  model.add(Dense(256,activation='relu'))
  model.add(Dense(1, activation='sigmoid'))
  return model


# Training

In [ ]:
models = [
    # {'name': 'effnetb7', 'func': create_EffNetB7, 'starting_fold': 1},
    {'name': 'cavanagh', 'func': Cavanagh, 'starting_fold': 1},
    {'name': 'dieleman', 'func': Dieleman, 'starting_fold': 1},
    {'name': 'resnet50', 'func': create_ResNet50, 'starting_fold': 1},
    # {'name': 'simple', 'func': SimpleModel, 'starting_fold': 1},
    # {'name': 'simpler', 'func': SimplerModel, 'starting_fold': 1},
]

if not os.path.isdir("gcs"):
  raise Exception("local GCS folder must be mounted!")

for model in models:
  model_name = model['name']
  model_factory = model['func']
  model_starting_fold = model['starting_fold']

  model_path = f"models/{IMAGE_SIZE}x{IMAGE_SIZE}/experiments/{EXPERIMENT_DESCRIPTION}/{model_name}"
  initial_model_path = f"models/{IMAGE_SIZE}x{IMAGE_SIZE}/initial_models/{model_name}"
  # Create initial model
  if not os.path.isdir(f"{LOCAL_GCP_PATH_BASE}/{initial_model_path}"):
    print("Creating new weights")
    strategy = reset_tpu()
    with strategy.scope():
      optimizer = optimizers.Adam(learning_rate= LEARNING_RATE, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
      model = model_factory()

      model.compile(
          loss=tf.keras.losses.BinaryCrossentropy(),
          steps_per_execution = 1,
          optimizer=optimizer,
          metrics=[tf.keras.metrics.BinaryAccuracy()])

      print(f"Saving initial model to {REMOTE_GCP_PATH_BASE}/{initial_model_path}")
      model.save(f"{REMOTE_GCP_PATH_BASE}/{initial_model_path}")

  for i in range(model_starting_fold, 11):
    tf.keras.backend.clear_session()
    strategy = reset_tpu()
    with strategy.scope():
    # with tf.device("/device:GPU:0"):
      optimizer = optimizers.Adam(learning_rate= LEARNING_RATE, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
      print(f"{model_name} FOLD {i}")

      best_epoch = 0
      best_loss = 1

      test_dataset = get_dataset(
          f"{REMOTE_GCP_PATH_BASE}/data/{IMAGE_SIZE}x{IMAGE_SIZE}/fold_{i}/test",
          TEST_BATCH_SIZE,
          seed = SEED,
          augment = False,
          shuffle = False,
          drop_remainder = True)

      if TPU:
        test_dataset = test_dataset.cache()

      print(f"Loading model from {REMOTE_GCP_PATH_BASE}/{initial_model_path}")
      model = tf.keras.models.load_model(f"{REMOTE_GCP_PATH_BASE}/{initial_model_path}")

      model.compile(
          loss=tf.keras.losses.BinaryCrossentropy(),
          steps_per_execution = 1,
          optimizer=optimizer,
          metrics=[tf.keras.metrics.BinaryAccuracy()])

      model.summary()

      for epoch in range(NUMBER_OF_EPOCHS):
        train_dataset = get_dataset(
            f"{REMOTE_GCP_PATH_BASE}/data/{IMAGE_SIZE}x{IMAGE_SIZE}/fold_{i}/train",
            TRAIN_BATCH_SIZE,
            seed = SEED + epoch,
            augment = True,
            shuffle = True,
            drop_remainder = False)

        history = model.fit(
          x= train_dataset,
          validation_data = test_dataset,
          epochs = 1,
          steps_per_epoch = 284672 // TRAIN_BATCH_SIZE,
          validation_steps = 31595 // TEST_BATCH_SIZE,
          verbose = 1,
          shuffle = False,
          workers= 32 if TPU else 1)

        last_loss = history.history['val_loss'][-1]
        if (last_loss < best_loss):
          print("Loss improved. Saving model.")
          best_epoch = epoch
          best_loss = last_loss
          model.save(f"{REMOTE_GCP_PATH_BASE}/{model_path}/best_loss/fold_{i}")


        if(epoch - EARLY_STOPPING_TOLERANCE == best_epoch):
          print("Early stopping")
          break

Running on TPU  grpc://10.31.17.98:8470
REPLICAS:  8
cavanagh FOLD 1
Loading model from gs://tomasmuzas-phd-datasets/SDSS/models/128x128/initial_models/cavanagh
Model: "Cavanagh"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 122, 122, 32)      4736      
                                                                 
 batch_normalization (BatchN  (None, 122, 122, 32)     128       
 ormalization)                                                   
                                                                 
 max_pooling2d (MaxPooling2D  (None, 61, 61, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 57, 57, 64)        51264     
                                                                 
 conv2d_2 (Conv2D)           

278/278 [==============================] - 81s 178ms/step - loss: 0.1910 - binary_accuracy: 0.9231 - val_loss: 0.1963 - val_binary_accuracy: 0.9230
Loss improved. Saving model.


278/278 [==============================] - 74s 179ms/step - loss: 0.1731 - binary_accuracy: 0.9302 - val_loss: 0.1625 - val_binary_accuracy: 0.9367
Loss improved. Saving model.


278/278 [==============================] - 68s 179ms/step - loss: 0.1622 - binary_accuracy: 0.9346 - val_loss: 0.1498 - val_binary_accuracy: 0.9402
Loss improved. Saving model.


278/278 [==============================] - 69s 177ms/step - loss: 0.1559 - binary_accuracy: 0.9373 - val_loss: 0.1496 - val_binary_accuracy: 0.9405
Loss improved. Saving model.


...

278/278 [==============================] - 63s 179ms/step - loss: 0.1040 - binary_accuracy: 0.9579 - val_loss: 0.1477 - val_binary_accuracy: 0.9438
Early stopping


Running on TPU  grpc://10.31.17.98:8470
REPLICAS:  8
cavanagh FOLD 5
Loading model from gs://tomasmuzas-phd-datasets/SDSS/models/128x128/initial_models/cavanagh
Model: "Cavanagh"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 122, 122, 32)      4736      
                                                                 
 batch_normalization (BatchN  (None, 122, 122, 32)     128       
 ormalization)                                                   
                                                                 
 max_pooling2d (MaxPooling2D  (None, 61, 61, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 57, 57, 64)        51264     
                                                                 
 conv2d_2 (Conv2D)           

278/278 [==============================] - 64s 180ms/step - loss: 0.1721 - binary_accuracy: 0.9310 - val_loss: 0.1536 - val_binary_accuracy: 0.9389
Loss improved. Saving model.


278/278 [==============================] - 64s 179ms/step - loss: 0.1551 - binary_accuracy: 0.9374 - val_loss: 0.1407 - val_binary_accuracy: 0.9440
Loss improved. Saving model.


278/278 [==============================] - 65s 180ms/step - loss: 0.1500 - binary_accuracy: 0.9397 - val_loss: 0.1442 - val_binary_accuracy: 0.9437


In [ ]:
# with tf.device("/device:GPU:0"):
train_dataset = get_dataset(
f"{REMOTE_GCP_PATH_BASE}/data/{IMAGE_SIZE}x{IMAGE_SIZE}/fold_1/train",
TRAIN_BATCH_SIZE,
seed = SEED + 1,
augment = True,
shuffle = True,
drop_remainder = False)

tfds.benchmark(train_dataset, batch_size=TRAIN_BATCH_SIZE)

1it [00:00, ?it/s]


************ Summary ************

Examples/sec (First included) 6285.04 ex/sec (total: 284672 ex, 45.29 sec)
Examples/sec (First only) 27.90 ex/sec (total: 1024 ex, 36.70 sec)
Examples/sec (First excluded) 33008.07 ex/sec (total: 283648 ex, 8.59 sec)


Exception ignored in: <function Executor.__del__ at 0x7f2f1be284d0>
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/tensorflow/python/eager/executor.py", line 46, in __del__
    self.wait()
  File "/usr/local/lib/python3.7/dist-packages/tensorflow/python/eager/executor.py", line 65, in wait
    pywrap_tfe.TFE_ExecutorWaitForAllPendingNodes(self._handle)
tensorflow.python.framework.errors_impl.OutOfRangeError: End of sequence


,duration,num_examples,avg
first+lasts,45.293556,284672,6285.044149
first,36.700264,1024,27.901707
lasts,8.593293,283648,33008.069569


In [ ]:
t = SimplerModel()
t.summary()

Model: "SimplerModel"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_6 (Conv2D)           (None, 123, 123, 128)     13952     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 61, 61, 128)      0         
 )                                                               
                                                                 
 conv2d_7 (Conv2D)           (None, 57, 57, 32)        102432    
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 28, 28, 32)       0         
 2D)                                                             
                                                                 
 conv2d_8 (Conv2D)           (None, 26, 26, 16)        4624      
                                                                 
 flatten_2 (Flatten)         (None, 10816)            